## Model Monitoring Pipeline

**Steps to be followed**
- Load the Validation data as Reference Data
- Inference data as current data
- Load model for Prediction purpose

In [2]:
pip install evidently


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 4.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.6/240.6 kB 6.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.5/71.5 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: watchdog
    Found existing installation: watchdog 2.1.6
    Uninstalling watchdog-2.1.6:
      Successfully uninstalled watchdog-2.1.6
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.

In [3]:
import pandas as pd
import pickle
from evidently.test_suite import TestSuite
from evidently.test_preset import DataStabilityTestPreset

from evidently.report import Report
from evidently.metric_preset import DataDriftPreset

import seaborn as sns
from evidently import ColumnMapping

from evidently.metric_preset import DataDriftPreset
from evidently.metric_preset import DataQualityPreset
from evidently.metric_preset import RegressionPreset
from evidently.metric_preset import ClassificationPreset
from evidently.metric_preset import TargetDriftPreset
from evidently.metric_preset import TextOverviewPreset

from evidently.metrics import *

from evidently.test_preset import NoTargetPerformanceTestPreset
from evidently.test_preset import DataStabilityTestPreset
from evidently.test_preset import DataQualityTestPreset
from evidently.test_preset import DataDriftTestPreset

from evidently.tests import *

from evidently.tests.base_test import generate_column_tests
from evidently.metrics.base_metric import generate_column_metrics

### Load Validation Data as Reference Data

In [5]:
df_reference = pd.read_csv('/Users/sumanmukherjee_admin/Desktop/Arpa/DS/physician_conversion_mlops-main/data/output/validation_data.csv')
print(df_reference.shape)
print('')
df_reference.head()
#/Users/sumanmukherjee_admin/Desktop/Arpa/DS/physician_conversion_mlops-main/data/output/validation_data.csv

(900, 33)



,Unnamed: 0,NPI_ID,HCP_ID,Age,Year_of_Experience,Number_of_Rx,Rx_last_1_Month,Rx_last_3_Month,Rx_last_6_Month,Rx_last_12_Month,...,Promotional_medscape_last_3_month,Promotional_medscape_last_6_month,Promotional_medscape_last_12_month,Sex_ M,Specialty_Oncology,Specialty_Pediatric,Specialty_Uro-oncology,HCO_Affiliation_Type_Contract,HCO_Affiliation_Type_Referral,TARGET
0,1916,6994026,HCP_1917,35,16,341,425,613,970,1134,...,24,37,46,True,False,False,True,False,True,0
1,196,2715035,HCP_197,77,10,126,140,198,318,539,...,7,9,13,False,False,False,False,False,False,0
2,3319,7534912,HCP_3320,90,45,1100,2103,2536,3920,4988,...,16,22,25,True,True,False,False,False,False,0
3,3466,7645107,HCP_3467,73,34,1100,1243,1552,1881,2937,...,29,35,66,False,False,False,False,False,True,1
4,2540,4857078,HCP_2541,54,54,1084,1456,1565,1804,2256,...,21,30,53,False,False,True,False,False,True,1


In [6]:
df_reference.drop(['Unnamed: 0'], axis=1, inplace = True)
df_reference.head()

,NPI_ID,HCP_ID,Age,Year_of_Experience,Number_of_Rx,Rx_last_1_Month,Rx_last_3_Month,Rx_last_6_Month,Rx_last_12_Month,Claims_last_1_Month,...,Promotional_medscape_last_3_month,Promotional_medscape_last_6_month,Promotional_medscape_last_12_month,Sex_ M,Specialty_Oncology,Specialty_Pediatric,Specialty_Uro-oncology,HCO_Affiliation_Type_Contract,HCO_Affiliation_Type_Referral,TARGET
0,6994026,HCP_1917,35,16,341,425,613,970,1134,24,...,24,37,46,True,False,False,True,False,True,0
1,2715035,HCP_197,77,10,126,140,198,318,539,320,...,7,9,13,False,False,False,False,False,False,0
2,7534912,HCP_3320,90,45,1100,2103,2536,3920,4988,341,...,16,22,25,True,True,False,False,False,False,0
3,7645107,HCP_3467,73,34,1100,1243,1552,1881,2937,316,...,29,35,66,False,False,False,False,False,True,1
4,4857078,HCP_2541,54,54,1084,1456,1565,1804,2256,199,...,21,30,53,False,False,True,False,False,True,1


### Load Inference Data as Current Data

In [7]:
df_current = pd.read_csv('/Users/sumanmukherjee_admin/Desktop/Arpa/DS/physician_conversion_mlops-main/data/output/Inference_data.csv')
print(df_current.shape)
print('')
df_current.head()

(500, 32)



,Unnamed: 0,NPI_ID,HCP_ID,Age,Year_of_Experience,Number_of_Rx,Rx_last_1_Month,Rx_last_3_Month,Rx_last_6_Month,Rx_last_12_Month,...,Promotional_medscape_last_1_month,Promotional_medscape_last_3_month,Promotional_medscape_last_6_month,Promotional_medscape_last_12_month,Sex_ M,Specialty_Oncology,Specialty_Pediatric,Specialty_Uro-oncology,HCO_Affiliation_Type_Contract,HCO_Affiliation_Type_Referral
0,1966,7552363,HCP_1967,60,44,1100,1297,1926,3288,4463,...,10,15,25,48,False,False,True,False,False,True
1,1376,4047649,HCP_1377,88,56,1100,1786,2959,5062,5420,...,10,19,33,44,True,True,False,False,False,False
2,881,5063276,HCP_882,54,33,175,350,467,564,583,...,20,22,43,75,True,False,False,False,False,False
3,4226,6628561,HCP_4227,85,10,730,1382,2526,4890,9192,...,12,15,27,52,True,True,False,False,False,True
4,4917,6186618,HCP_4918,61,60,1100,1717,1809,2926,4006,...,14,15,24,45,True,False,False,True,True,False


In [8]:
df_current.drop(['Unnamed: 0'], axis=1, inplace = True)
df_current.head()

,NPI_ID,HCP_ID,Age,Year_of_Experience,Number_of_Rx,Rx_last_1_Month,Rx_last_3_Month,Rx_last_6_Month,Rx_last_12_Month,Claims_last_1_Month,...,Promotional_medscape_last_1_month,Promotional_medscape_last_3_month,Promotional_medscape_last_6_month,Promotional_medscape_last_12_month,Sex_ M,Specialty_Oncology,Specialty_Pediatric,Specialty_Uro-oncology,HCO_Affiliation_Type_Contract,HCO_Affiliation_Type_Referral
0,7552363,HCP_1967,60,44,1100,1297,1926,3288,4463,244,...,10,15,25,48,False,False,True,False,False,True
1,4047649,HCP_1377,88,56,1100,1786,2959,5062,5420,97,...,10,19,33,44,True,True,False,False,False,False
2,5063276,HCP_882,54,33,175,350,467,564,583,193,...,20,22,43,75,True,False,False,False,False,False
3,6628561,HCP_4227,85,10,730,1382,2526,4890,9192,37,...,12,15,27,52,True,True,False,False,False,True
4,6186618,HCP_4918,61,60,1100,1717,1809,2926,4006,37,...,14,15,24,45,True,False,False,True,True,False


### Load the Classifier

In [11]:
# Load the model
with open("/Users/sumanmukherjee_admin/Desktop/Arpa/DS/physician_conversion_mlops-main/data/output/model_file/physician_conversion.pkl", "rb") as f:
    conversion_classifer = pickle.load(f)

/Users/sumanmukherjee_admin/anaconda3/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [20:57:03] WARNING: /Users/runner/work/xgboost/xgboost/src/gbm/../common/error_msg.h:80: If you are loading a serialized model (like pickle in Python, RDS in R) or
configuration generated by an older version of XGBoost, please export the model by calling
`Booster.save_model` from that version first, then load it back in current version. See:

    https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html

for more details about differences between saving model and serializing.

  warnings.warn(smsg, UserWarning)


In [12]:
ref_prediction = conversion_classifer.predict(df_reference.drop(drop_id_col_with_target_list, axis=1))
current_prediction = conversion_classifer.predict(df_current.drop(drop_id_col_list, axis=1))

NameError: name 'drop_id_col_with_target_list' is not defined

In [ ]:
#id col not to be considered while training
#drop_id_col_with_target_list = ['NPI_ID', 'HCP_ID','TARGET']
df_reference['prediction'] = conversion_classifer.predict(df_reference.drop(drop_id_col_list + ['TARGET'], axis=1))
df_reference['prediction'].value_counts()

prediction
0    716
1    184
Name: count, dtype: int64

In [ ]:
#id col not to be considered while training
drop_id_col_list = ['NPI_ID', 'HCP_ID']

In [ ]:
df_current['prediction'] = conversion_classifer.predict(df_current.drop(drop_id_col_list, axis=1))
df_current['TARGET'] = df_current['prediction']

In [ ]:
df_current['prediction'].value_counts()

prediction
0    381
1    119
Name: count, dtype: int64

In [ ]:
df_current.columns

Index(['NPI_ID', 'HCP_ID', 'Age', 'Number_of_Rx', 'Rx_last_1_Month',
       'Rx_last_3_Month', 'Rx_last_6_Month', 'Rx_last_12_Month',
       'Claims_last_1_Month', 'Claims_last_3_Month',
       'Procedures_chemo_last_1_month', 'Procedures_chemo_last_3_month',
       'Procedures_chemo_last_6_month', 'Procedures_chemo_last_12_month',
       'Procedures_radio_last_1_month', 'Procedures_Immuno_last_6_month',
       'Procedures_Immuno_last_12_month', 'Procedures_Biopsy_last_3_month',
       'Procedures_Biopsy_last_6_month', 'Procedures_Biopsy_last_12_month',
       'Promotional_medscape_last_1_month',
       'Promotional_medscape_last_3_month',
       'Promotional_medscape_last_6_month', 'F2F_visit',
       'F2F_visit_last_1_month', 'F2F_visit_last_3_month',
       'VRC_visit_last_3_month', 'VRC_visit_last_6_month',
       'VRC_visit_last_12_month', 'HCO_Affiliation_Type_Contract',
       'HCO_Affiliation_Type_Referral', 'prediction', 'TARGET'],
      dtype='object')

In [ ]:
numerical_features = ['Age', 'Number_of_Rx', 'Rx_last_1_Month',
       'Rx_last_3_Month', 'Rx_last_6_Month', 'Rx_last_12_Month',
       'Claims_last_1_Month', 'Claims_last_3_Month',
       'Procedures_chemo_last_1_month', 'Procedures_chemo_last_3_month',
       'Procedures_chemo_last_6_month', 'Procedures_chemo_last_12_month',
       'Procedures_radio_last_1_month', 'Procedures_Immuno_last_6_month',
       'Procedures_Immuno_last_12_month', 'Procedures_Biopsy_last_3_month',
       'Procedures_Biopsy_last_6_month', 'Procedures_Biopsy_last_12_month',
       'Promotional_medscape_last_1_month',
       'Promotional_medscape_last_3_month',
       'Promotional_medscape_last_6_month', 'F2F_visit',
       'F2F_visit_last_1_month', 'F2F_visit_last_3_month',
       'VRC_visit_last_3_month', 'VRC_visit_last_6_month',
       'VRC_visit_last_12_month']
categorical_features = ['HCO_Affiliation_Type_Contract',
       'HCO_Affiliation_Type_Referral'] 

### Model Monitoring
- Data/Feature Drift
- Model Drift (if possible)

In [ ]:
column_mapping = ColumnMapping()

column_mapping.numerical_features = numerical_features
column_mapping.categorical_features = categorical_features


In [ ]:
# data_quality_report = Report(metrics=[
#     DataQualityPreset(),
# ])

# data_quality_report.run(reference_data=None, current_data=df_current)

In [ ]:
data_quality_report = Report(metrics=[
    DataQualityPreset(),
])

data_quality_report.run(reference_data=df_reference, current_data=df_current)

C:\Users\harshit.sundriyal\AppData\Roaming\Python\Python311\site-packages\evidently\utils\visualizations.py:553: RuntimeWarning:

Converting input from bool to <class 'numpy.uint8'> for compatibility.

C:\Users\harshit.sundriyal\AppData\Roaming\Python\Python311\site-packages\evidently\utils\visualizations.py:554: RuntimeWarning:

Converting input from bool to <class 'numpy.uint8'> for compatibility.

C:\Users\harshit.sundriyal\AppData\Roaming\Python\Python311\site-packages\evidently\utils\visualizations.py:558: RuntimeWarning:

Converting input from bool to <class 'numpy.uint8'> for compatibility.

C:\Users\harshit.sundriyal\AppData\Roaming\Python\Python311\site-packages\evidently\utils\visualizations.py:553: RuntimeWarning:

Converting input from bool to <class 'numpy.uint8'> for compatibility.

C:\Users\harshit.sundriyal\AppData\Roaming\Python\Python311\site-packages\evidently\utils\visualizations.py:554: RuntimeWarning:

Converting input from bool to <class 'numpy.uint8'> for compat

In [ ]:
data_quality_report.show()

In [ ]:
classification_performance_report = Report(metrics=[
    ClassificationPreset(),
])

classification_performance_report.run(reference_data=df_reference, current_data=df_current)